<font size="3">

## Sistema di auto-completamento

Il progetto consiste nel realizzare un sistema di auto completamento di parole che rappresentano delle skill tecniche.

Il sistema suggerisce all'utente la skill con la relazione maggiore rispetto a quelle già inserite, è anche in grado di suggerire il completamento di una skill, mentre l'utente la sta inserendo, basandosi sullo stesso principio.

### Workflow

1. Se l'utente non ha ancora inserito nessuna skills (context = []), allora si suggeriscono le skill più simili all'input inserito


2. Per le successive stringe (context $\neq$ []) si considerano le skill inserite fino a quel momento: 
    - si calcola la similarità dell'input inserito dall'utente e le skills non nel context:
    
    $$A = \{(s, sim(i, s)) \: | \: s \in S \: \& \: s \notin C\}$$<br>
    - per ogni skills non nel context, si calcola la media delle similarità tra quella skills e quelle nel context:
    
    $$B = \{(s, sim_{avg}(s, C)) \: | \: s \in S \: \& \: s \notin C\}$$
    
    dove $sim_{avg}(s, C) = \frac{\sum_{c \in C}sim(s, c)}{|C|}$
    <br>
    - per ogni skills non nel context si calcola la similarità complessiva come:
    
    $$sim_s = \alpha \cdot sim_A + (1-\alpha) \cdot sim_B \qquad \forall s \in S \:\&\: s \notin C$$<br>
    dove $(s, sim_A) \in A$ e $(s, sim_B) \in B$


3. Si suggeriscono anche skills senza tener conto dell'input inserito e considerando solo il context, in questo caso le similarità sono calcolate utilizzando solo $B$.
</font>

In [1]:
import numpy as np                                                 # For some operations
import asyncio                                                     # For delay suggests function
from gensim.models import FastText                                 # Load FastText vector model
from IPython.core.display import display                           # Display UI components
from ipywidgets import Layout, Textarea, HBox, VBox, Label, Button, interact# Widgets and components for UI

In [2]:
class FormUI:
    """
    Class for UI, manage and show the UI components as the text area na buttons.

    
    Modules
    -------
    IPython.core.display (IPython version 7.13.0)
    ipywidgets.Layout (ipywidgets version 7.5.1)
    ipywidgets.Textarea (ipywidgets version 7.5.1)
    ipywidgets.HBox (ipywidgets version 7.5.1)
    ipywidgets.VBox (ipywidgets version 7.5.1)
    ipywidgets.Label (ipywidgets version 7.5.1)
    ipywidgets.Button (ipywidgets version 7.5.1)

    Attributes
    ----------
    layout_button : ipywidgets.Layout
        The buttons layout
    layout_buttonBox : ipywidgets.Layout
        The buttons box layout
    layout_textArea : ipywidgets.Layout
        The text area layout
    layout_textAreaBox : ipywidgets.Layout
        The text area box layout
    suggest_buttons : list
        List of buttons for suggestions based context and input
    suggest_buttons_context : list
        List of buttons for suggestions based only context
    textArea : ipywidgets.TextArea
        The text area widget
    suggests : ipywidgets.HBox
        Horizontal box for suggest_buttons
    suggests_context : ipywidgets.HBox
        Horizontal box for suggest_buttons_context
    form : ipywidgets.VBox
        Vertical box for the components

    Methods
    -------
    __init__()
        Constructor of the class, it initializes the various components
    show_form(sound=None)
        Display the form
    init_button(description, button_style, tooltip)
        Inizilize button with description and text for tooltip
    close_suggest_buttons()
        Closes the buttons on the suggest_buttons list
    close_suggest_buttons_context()
        Closes the buttons on the suggest_buttons_context list
    """
    
    def __init__(self):
        """
        Constructor of the class, it initializes the various components.
        
        """
        
        # Layouts
        self.layout_button = Layout(height='50px', width='25%')
        self.layout_buttonBox = Layout(height='70px')
        self.layout_textArea = Layout(height='100px', width='auto')
        self.layout_textAreaBox = Layout(margin='0 0 20px 0')
        
        # Lists button
        self.suggest_buttons = list()
        self.suggest_buttons_context = list()
        
        # Text area
        self.textArea = Textarea(
            value='',
            placeholder='Type skills',
            disabled=False,
            tooltip='Your skills',
            layout=self.layout_textArea
        )
        
        # Box for suggest_buttons
        self.suggests = HBox(self.suggest_buttons, layout=self.layout_buttonBox)
        
        # Box for suggest_buttons_context
        self.suggests_context = HBox(self.suggest_buttons_context, layout=self.layout_buttonBox)
        
        
        # Box for all components
        self.form = VBox([VBox([Label(value='Skills:'), self.textArea], layout=self.layout_textAreaBox), 
                          VBox([Label(value='Suggests:'), self.suggests]),
                          VBox([Label(value='Similar skills:'), self.suggests_context])])
        
    def show_form(self):
        """
        Display the UI form.
        
        """
        display(self.form)
        
        
    
    def init_button(self, description, button_style, tooltip):
        '''
        Inizilize button with description and text for tooltip.

        Parameters
        ----------
        description : str
            The button's description text
        button_style : str
            The button's style, can be 'success', 'info', 'warning', 'danger' or ''
        tooltip : str
            The button's tooltip text
        
        Returns
        -------
        ipywidgets.Button
            The new button
        
        '''
        
        # Create button
        b = Button(
            description=description,
            disabled=False,
            button_style=button_style,  # 'success', 'info', 'warning', 'danger' or ''
            tooltip=tooltip,
            layout=self.layout_button
        )

        return b
    
    
    def close_suggest_buttons(self):
        '''
        Closes the buttons on the suggest_buttons list.
        
        '''
        for w in self.suggest_buttons:
            w.close()
        self.suggest_buttons = list()
    
    
    def close_suggest_buttons_context(self):
        '''
        Closes the buttons on the suggest_buttons_context list.
        
        '''
        for w in self.suggest_buttons_context:
            w.close()
        self.suggest_buttons_context = list()
   
    

In [3]:
 class Delay:
    """
    Class for manage text area delay on user interaction.


    Modules
    -------
    asyncio

    Attributes
    ----------
    timeout : float
        The delay time
    callback : Python function reference
        The collback to be performed after the timeout
    task : Task
        The task which is associated with the callback

    Methods
    -------
    __init__(timeout, callback)
        Constructor of the class, it initializes the various components
    job()
        Asybc function, after delay, performs the job which is associated with the callback
    cancel():
        Cancel the task
    """
    
    def __init__(self, timeout, callback):
        '''
        Constructor of the class, it initializes the various components.
        
        Parameters
        ----------
        timeout : float
            The delay time
        callback : Python function reference
            The collback to be performed after the timeout
        
        '''
        self.timeout = timeout
        self.callback = callback
        self.task = asyncio.ensure_future(self.job())

    async def job(self):
        '''
        Asybc function, after delay, performs the job which is associated with the callback.
        
        '''
        await asyncio.sleep(self.timeout)
        self.callback()

    def cancel(self):
        '''
        Cancel the task.
        
        '''
        self.task.cancel()

In [77]:
class AutoCompleteManager:
    
    """
    Class for UI, manage and show the UI components as the text area na buttons.


    Modules
    -------
    numpy (version 1.16.6)
    gensim.models.FastText (gensim version 3.8.1)

    Attributes
    ----------
    skills_list : list
        The list of skills
    model : gensim.models.FastText
        The gensim FastText model
    alpha : float
        The alpha parameter for similarity weighing
    num_suggests : int
        The number of suggestion to show
    FormUI : FormUI
        The UI form

    Methods
    -------
    __init__(skills_list, model, alpha=0.8, num_suggests=4)
        Constructor of the class, it initializes the various components
    get_skills_input_similarity(word, context)
        Compute the similarity between the user's input and the skills that are not in context
    get_skills_context_similarity(context)
        Compute the similarity between the context skills and other skills
    get_best_similarity_skill(new_input, context)
        Compute the overall similarity and return the most similarity skills
    suggest_interest_skill(context)
        Suggests the skills most similar to the context
    add_skill(btn_object)
        Add the skill chosen by the user to the text area
    debounce(wait):
        Decorator that will postpone a function's execution until after "wait" seconds
    suggests_manager(textArea)
        Listener function for text area user's interaction
    show_form()
        Show the UI form and bind the text area to its listener suggests_manager
    """

    
    def __init__(self, skills_list, model, alpha=0.8, num_suggests=4):
        '''
        Constructor of the class, it initializes the various components.
        
        Parameters
        ----------
        skills_list : list
            The list of skills
        model : gensim.models.FastText
            The gensim FastText model
        alpha : float, optional
            The alpha parameter for similarity weighing (default is 0.8)
        num_suggests : int, optional
            The number of suggestion to show (default is 4)

        Raises
        ------
        AssertionError
            If the alpha parameter must be greater than 0 and less than 1.
        AssertionError
            If the model parameter must be a FastText instance.
        AssertionError
            If the num_suggests parameter must be a int.
        
        '''
        # Input assert check
        assert 0 <= alpha <= 1, "The alpha parameter must be greater than 0 and less than 1"
        assert isinstance(model, FastText), "The model parameter must be a FastText instance"
        assert isinstance(num_suggests, int), "The num_suggests parameter must be a int"
        
        # Set the parameters
        self.alpha = alpha
        self.num_suggests = num_suggests
        self.skills_list = skills_list
        self.model = model
        self.FormUI = FormUI()
        
    
    def get_skills_input_similarity(self, word, context):
        '''
        Compute the similarity between the user's input and the skills that are not in context.
        
        Parameters
        ----------
        word : str
            A single word
        context : list
            The skills context
        
        Returns
        -------
        dict
            A dictionary with keys the skills and values their similarity with input word        
        
        '''
        return {k: self.model.wv.similarity(word, k) for k in self.skills_list if k not in context}
        
        
    def get_skills_context_similarity(self, context):
        '''
        Compute the similarity between the context skills and other skills.
        
        Parameters
        ----------
        context : list
            The skills context
        
        Returns
        -------
        dict
            A dictionary with keys the skills and values the mean of similarity between the key
            and the skills in the context        
        
        '''
        res = dict()
        for s in self.skills_list:
            if s not in context:
                res[s] = np.array([self.model.wv.similarity(s, c) for c in context]).mean()
        return res
    
    
    def get_best_similarity_skill(self, new_input, context):
        '''
        Compute the overall similarity and return the most similarity skills.
        
        Parameters
        ----------
        new_input : str
            The user's input
        context : list
            The skills context
        
        Returns
        -------
        dict
            A dictionary with the first num_suggests skills more similar to both the context and the 
            user's input       
        
        '''
        # Similarity between context and input
        sim_with_input = self.get_skills_input_similarity(new_input, context)
        
        # If context is empty return only input similarity
        if context == []:
            return {k: v for k, v in sorted(sim_with_input.items(), key=lambda item: -item[1])[:self.num_suggests]}
        
        res_sim = dict()
        
        # Average similarity between every skill and the skills in the context
        sim_with_context = self.get_skills_context_similarity(context)
        
        # Overall similarity
        for skill in sim_with_context.keys():
            res_sim[skill] = self.alpha*sim_with_input[skill]+(1-self.alpha)*sim_with_context[skill]
        
        return {k: v for k, v in sorted(res_sim.items(), key=lambda item: -item[1])[:self.num_suggests]}
        
        
    def suggest_interest_skill(self, context):
        '''
        Suggests the skills most similar to the context, create and show their buttons.
        
        Parameters
        ----------
        context : list
            The skills context
             
        '''
        
        # Get similarity
        similarity_context = self.get_skills_context_similarity(context)
        similarity_context = {k: v for k, v in sorted(similarity_context.items(), 
                                                      key=lambda item: -item[1])[:self.num_suggests]}
        
        # Create buttons
        for index, value in similarity_context.items():
            b = self.FormUI.init_button(description=f'{index} - \n{round(value * 100, 2)}%',
                                        button_style='info',
                                        tooltip=f'{index} - {round(value * 100, 2)}%')
            
            b.on_click(self.add_skill)
            self.FormUI.suggest_buttons_context.append(b)
        
        # Add buttons to form
        self.FormUI.suggests_context.children=tuple(self.FormUI.suggest_buttons_context)
    
    
    def add_skill(self, btn_object):
        '''
        Add the skill chosen by the user to the text area.
        
        Parameters
        ----------
        btn_object : ipywidgets.Button
            The pressed button    
        
        '''
        # Get the text area values
        skill_list = self.FormUI.textArea.value.split('; ')
        
        # Add new skill to the text area value
        skill_list[-1] = btn_object.description.split(" -")[0]
        new_contest = '; '.join(skill_list) + '; '
        self.FormUI.textArea.value = new_contest
        
        # Remove the buttons with old suggest
        self.FormUI.close_suggest_buttons()
        self.FormUI.close_suggest_buttons_context()
        
        # Compute the similarity with only the context
        self.suggest_interest_skill(skill_list)
        
    
    def debounce(wait):
        ''' 
        Decorator that will postpone a function's execution until after "wait" seconds
        have elapsed since the last time it was invoked.
        
        ref: https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html#Debouncing
        
        Parameters
        ----------
        wait : float
            The delay timeout
        
        '''
        def decorator(fn):
            timer = None
            def debounced(*args, **kwargs):
                
                # Use locan inside inner function
                nonlocal timer
                
                # Wrapper for the function to pass at Time class
                def function_wrapper():
                    fn(*args, **kwargs)
                    
                # Remove old timer
                if timer is not None:
                    timer.cancel()
                
                # Create Time instance for delay and then pass the call_it wrapper
                timer = Delay(wait, function_wrapper)
            return debounced
        return decorator
    
        
    @debounce(0.3)
    def suggests_manager(self, textArea):
        '''
        Listener function for text area user's interaction.
        
        Parameters
        ----------
        textArea : ipywidgets.TextArea
            The text area widget   
        
        '''
        # Remove old buttons
        self.FormUI.close_suggest_buttons()
        
        # Get the context
        old_input = self.FormUI.textArea.value.split('; ')
        context = old_input[:-1]
        # Check if the last string is a skills or a new input
        if old_input[-1] in self.skills_list:
            context.append(old_input[-1])
        
        # Get the new input
        new_input = textArea['new'].split('; ')[-1]
        
        
        if len(new_input)>3:
            self.FormUI.close_suggest_buttons()
            
            best_similarity = self.get_best_similarity_skill(new_input, context)
            
            for index, value in best_similarity.items():
                b = self.FormUI.init_button(description=f'{index} - \n{round(value * 100, 2)}%',
                                            button_style='success',
                                            tooltip=f'{index} - {round(value * 100, 2)}%')
                b.on_click(self.add_skill)
                self.FormUI.suggest_buttons.append(b)
        
        if context == []:
            self.FormUI.close_suggest_buttons_context()
            
        # Show buttons with new suggest
        self.FormUI.suggests.children=tuple(self.FormUI.suggest_buttons)
        
        
    
    def show_form(self):
        '''
        Show the UI form and bind the text area to its listener suggests_manager.
        
        '''
        self.FormUI.show_form()
        self.FormUI.textArea.observe(self.suggests_manager, names='value')
    

In [79]:
# System deployment, the system deployment is simulated with the various settings:

import pandas as pd
# The list of skills
skills_list = pd.read_excel("data/2020_06_09 Allocation to ONET.xlsx")['escoskill_level_3']

# Load embeddings vector
vectors = FastText.load_fasttext_format("data/ft_vectors_cbow_50_10_0_05.bin")

# The alpha parameter for the final similarity
alpha = 0.8

# The number of suggests to show
num_suggests = 4

/home/simone/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  


In [80]:
# Initialization and display of the user interface
t = AutoCompleteManager(skills_list, vectors, alpha, num_suggests)
t.show_form()